In [1]:
%load_ext autoreload
%autoreload 2

import logging

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import defi.dao_cross.dao_cross as ddcrdacr
import defi.dao_cross.order as ddacrord

In [2]:
hdbg.init_logger(verbosity=logging.DEBUG)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-2795fa9f-2944-48df-b7ab-b38c8892665a.json'
DEBUG Effective logging level=10
DEBUG Shut up 1 modules: asyncio
DEBUG > (cd . && cd "$(git rev-parse --show-toplevel)/.." && (git rev-parse --is-inside-work-tree | grep -q true)) 2>&1
DEBUG > (git rev-parse --show-toplevel) 2>&1
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.1' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
DEBUG > (git branch --show-current) 2>&1
DEBUG > (git rev-parse --short HEAD) 2>&1
DEBUG > (git log --date=local --oneline --graph --date-order --decorate --pretty=format:'%h %<(8)%aN%  %<(65)%s (%>(14)%ar) %ad %<(10)%d' -3)

In [3]:
buy_order1 = ddacrord.Order("ETH", "BTC", "buy", 10, 0.1, 1, 1)
buy_order2 = ddacrord.Order("ETH", "BTC", "buy", 9, 0.15, 2, 2)
buy_order3 = ddacrord.Order("ETH", "BTC", "buy", 10, 0.15, 3, 3)
buy_orders = [buy_order1, buy_order2, buy_order3]
display(buy_orders)

[base_token=ETH quote_token=BTC action=buy quantity=10 limit_price=0.1 timestamp=2023-04-01 17:51:20.783205+00:00 deposit_address=1,
 base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=0.15 timestamp=2023-04-01 17:51:20.783341+00:00 deposit_address=2,
 base_token=ETH quote_token=BTC action=buy quantity=10 limit_price=0.15 timestamp=2023-04-01 17:51:20.783384+00:00 deposit_address=3]

In [4]:
sell_order1 = ddacrord.Order("ETH", "BTC", "sell", 11, 0.1, -1, -1)
sell_order2 = ddacrord.Order("ETH", "BTC", "sell", 10, 0.15, -2, -2)
sell_order3 = ddacrord.Order("ETH", "BTC", "sell", 9, 0.15, -3, -3)
sell_orders = [sell_order1, sell_order2, sell_order3]
display(sell_orders)

[base_token=ETH quote_token=BTC action=sell quantity=11 limit_price=0.1 timestamp=2023-04-01 17:51:20.816836+00:00 deposit_address=-1,
 base_token=ETH quote_token=BTC action=sell quantity=10 limit_price=0.15 timestamp=2023-04-01 17:51:20.816931+00:00 deposit_address=-2,
 base_token=ETH quote_token=BTC action=sell quantity=9 limit_price=0.15 timestamp=2023-04-01 17:51:20.816974+00:00 deposit_address=-3]

In [5]:
ddacrord.get_random_order()

base_token=ETH quote_token=BTC action=buy quantity=4 limit_price=inf timestamp=2023-04-01 17:51:20.853091+00:00 deposit_address=1

# Order matching

## Example 1

In [6]:
clearing_price = 0.14
transfers_df = ddcrdacr.match_orders(buy_orders + sell_orders, clearing_price)

DEBUG Order not eligible for matching due to limit
DEBUG Order not eligible for matching due to limit
DEBUG Order not eligible for matching due to limit
DEBUG buy_heap=[base_token=ETH quote_token=BTC action=buy quantity=10 limit_price=0.15 timestamp=2023-04-01 17:51:20.783384+00:00 deposit_address=3, base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=0.15 timestamp=2023-04-01 17:51:20.783341+00:00 deposit_address=2]
DEBUG sell_heap=[base_token=ETH quote_token=BTC action=sell quantity=11 limit_price=0.1 timestamp=2023-04-01 17:51:20.816836+00:00 deposit_address=-1]


In [7]:
eth_transfers = transfers_df[transfers_df["token"] == "ETH"]
display(eth_transfers)

,token,amount,from,to
0,ETH,9.0,-1,2
2,ETH,2.0,-1,3


In [8]:
# Transferred ETH (in ETH).
eth_transfers["amount"].sum()

11.0

In [9]:
btc_transfers = transfers_df[transfers_df["token"] == "BTC"]
display(btc_transfers)

,token,amount,from,to
1,BTC,1.26,2,-1
3,BTC,0.28,3,-1


In [10]:
# Transferred BTC (in BTC).
btc_transfers["amount"].sum()

1.5400000000000003

In [11]:
# BTC transferred divided by the ETH/BTC price (quoted in BTC per ETH)
# returns amount of transferred ETH.
btc_transfers["amount"].sum() / clearing_price

11.0

## Example 2

In [12]:
orders = []
for idx in range(0, 10):
    order = ddacrord.get_random_order(idx)
    orders.append(order)

In [13]:
orders

[base_token=ETH quote_token=BTC action=sell quantity=6 limit_price=-inf timestamp=2023-04-01 17:51:21.081703+00:00 deposit_address=-3,
 base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-04-01 17:51:21.081761+00:00 deposit_address=2,
 base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-04-01 17:51:21.081784+00:00 deposit_address=0,
 base_token=ETH quote_token=BTC action=sell quantity=4 limit_price=-inf timestamp=2023-04-01 17:51:21.081805+00:00 deposit_address=-3,
 base_token=ETH quote_token=BTC action=sell quantity=8 limit_price=-inf timestamp=2023-04-01 17:51:21.081824+00:00 deposit_address=-2,
 base_token=ETH quote_token=BTC action=buy quantity=7 limit_price=inf timestamp=2023-04-01 17:51:21.081843+00:00 deposit_address=3,
 base_token=ETH quote_token=BTC action=sell quantity=4 limit_price=-inf timestamp=2023-04-01 17:51:21.081861+00:00 deposit_address=-1,
 base_token=ETH quote_token=BTC action=buy quantity=7 limit_pric

In [14]:
transfers_df_2 = ddcrdacr.match_orders(orders, 0.1)
display(transfers_df_2)

DEBUG buy_heap=[base_token=ETH quote_token=BTC action=buy quantity=6 limit_price=inf timestamp=2023-04-01 17:51:21.081914+00:00 deposit_address=3, base_token=ETH quote_token=BTC action=buy quantity=7 limit_price=inf timestamp=2023-04-01 17:51:21.081878+00:00 deposit_address=0, base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-04-01 17:51:21.081784+00:00 deposit_address=0, base_token=ETH quote_token=BTC action=buy quantity=9 limit_price=inf timestamp=2023-04-01 17:51:21.081761+00:00 deposit_address=2, base_token=ETH quote_token=BTC action=buy quantity=7 limit_price=inf timestamp=2023-04-01 17:51:21.081843+00:00 deposit_address=3]
DEBUG sell_heap=[base_token=ETH quote_token=BTC action=sell quantity=2 limit_price=-inf timestamp=2023-04-01 17:51:21.081896+00:00 deposit_address=-2, base_token=ETH quote_token=BTC action=sell quantity=4 limit_price=-inf timestamp=2023-04-01 17:51:21.081861+00:00 deposit_address=-1, base_token=ETH quote_token=BTC action=sell q

,token,amount,from,to
0,ETH,7.0,-2,3
1,BTC,0.7,3,-2
2,ETH,1.0,-2,2
3,BTC,0.1,2,-2
4,ETH,6.0,-3,2
5,BTC,0.6,2,-3
6,ETH,2.0,-3,2
7,BTC,0.2,2,-3
8,ETH,2.0,-3,0
9,BTC,0.2,0,-3
